In [ ]:
import pandas as pd
import numpy
df = pd.read_pickle('../Practice/4990project/Yelp/Yelp_WebApp/WebApp_starter/df_user_for_sql')
df_business = pd.read_pickle('../Practice/4990project/Yelp/Yelp_WebApp/WebApp_starter/business_detail_for _sql')
GM = numpy.load('../Practice/4990project/Yelp/Yelp_WebApp/WebApp_starter/diffsion')
df_user = pd.read_pickle('../Practice/4990project/Yelp/Yelp_WebApp/WebApp_starter/df_user_to_sql')

def cosine(x, y):
    z = 1. - numpy.dot(x, y) / (numpy.linalg.norm(x, ord=2) * numpy.linalg.norm(y, ord=2))
    return z

In [ ]:
import MySQLdb

#connect to db
db = MySQLdb.connect("localhost","root","zfc17221658","test" )
cursor = db.cursor()

In [ ]:
# User_information table
sql = """CREATE TABLE user6 (
        user_id CHAR(100),  
        city CHAR(100),
        fans INT,
        name CHAR(100),
        review_count INT,
        useful INT)"""
cursor.execute(sql)

for i in xrange(len(df)):
    cursor.execute("""INSERT INTO user6 VALUES (%s, %s, %s, %s, %s, %s)""",(df.values[i][0],
                                                                            df.values[i][1], 
                                                                            df.values[i][2], 
                                                                            df.values[i][3], 
                                                                            df.values[i][4],
                                                                            df.values[i][5]))

db.commit()

In [ ]:
# Business_detail table
categories = df_business.columns.str.replace('&', 'and').str.replace('/', '_').str.replace('-', '_'). \
            str.replace('(', '').str.replace(')', '').str.replace("'", '').values[:-5]

sql = """CREATE TABLE business_detail8 (
        address char(255),
        business_id char(30),
        business_city char(100),
        business_name char(100)
        )"""
cursor.execute(sql)

for i in categories:
    cursor.execute("""alter table business_detail8 add %s int""" %(i))

cursor.execute("""alter table business_detail8 add categories__Pop_up_Shops int""")
A = df_business.values
for i in xrange(len(df_business)):
    if i == 809:
        continue
    if i == 1102:
        continue
    a = numpy.array([])
    a = numpy.append(numpy.append(numpy.append(a, A[i][-5:-1]), A[i][:-5]), A[i][-1])
    b = numpy.array2string(a).replace('[', '(').replace(']', ')').replace(' ', ',')
    if b[1] == ',':
        b = b[0]+b[2:]
    cursor.execute("""INSERT INTO business_detail8 VALUES %s;""" %(b))
    
#     print(i)
db.commit()

In [ ]:
# GM diffusion operator table

sql = """CREATE TABLE GM (
        c0 float(10)
        )"""
cursor.execute(sql)

for i in xrange(1,369):
    cursor.execute("""alter table GM add c%s float(10)""" %(i))
db.commit()

GM[numpy.isnan(GM)] = 0.
for i in xrange(len(GM)):
    cursor.execute("""INSERT INTO GM VALUES %s;""" %(b))
    
    print(i)
db.commit()

In [ ]:
# User_detail table
categories = df_user.columns.str.replace('&', 'and').str.replace('/', '_').str.replace('-', '_'). \
            str.replace('(', '').str.replace(')', '').str.replace("'", '').values[1:-1]

sql = """CREATE TABLE user_detail (
        user_id char(255),
        city char(200)
        )"""
cursor.execute(sql)

for i in categories:
    cursor.execute("""alter table user_detail add %s int""" %(i))
db.commit()

A = df_user.values
for i in xrange(len(df_user)):
    a = numpy.array([])
    a = numpy.append(numpy.append(numpy.append(a, A[i][0]), A[i][-1]), A[i][1:-1])
    b = numpy.array2string(a).replace('[', '(').replace(']', ')').replace(' ', ',')

    cursor.execute("""INSERT INTO user_detail VALUES %s;""" %(b))
    
    print(i)
db.commit()

In [ ]:
# User_recommendation table
sql = """CREATE TABLE user_recommend5 (
        self_user int,
        self_user_id char(100),
        user1 int,  
        user2 int,
        user3 INT,
        user1_id CHAR(100),
        user2_id CHAR(100),
        user3_id CHAR(100)
        )
        """
cursor.execute(sql)
X = df_user.drop(['user_id', 'city'], axis=1).values
id_ = df_user.user_id.values
N = len(df)
for nu in xrange(N):
    self_user = nu
    self_user_id = id_[nu]
    self_user_habit = X[nu, :]
    user_cosine = []
    for i in xrange(N):
        if i == nu:
            user_cosine.append(1)
            continue
        if numpy.count_nonzero(X[i, :]) == 0:
            user_cosine.append(1)
            continue
        user_cosine.append(cosine(X[i, :], self_user_habit))
    user1, user2, user3 = numpy.array(user_cosine).argsort()[:3][::-1]
    user1_id, user2_id, user3_id = id_[[user1, user2, user3]]
    cursor.execute("""INSERT INTO user_recommend5 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",
                   (self_user,
                    self_user_id, 
                    user1,
                    user2,
                    user3,
                    user1_id,
                    user2_id,
                    user3_id))
    if nu % 100 ==0:
        print nu

db.commit()


In [1]:
# Business_recommendation table
sql = """CREATE TABLE business_recommend3 (
        user_id CHAR(100),  
        user_number INT,
        business1 CHAR(100),
        business2 CHAR(100),
        business3 CHAR(100),
        categories1 CHAR(100),
        categories2 CHAR(100),
        categories3 CHAR(100))"""
cursor.execute(sql)

db.commit()

u_h = df_user.values[:, 1:-2]
b_h = df_business.values[:, :-5]
categories = df_user.columns.values[1:-2]
uid = df_user.user_id.values
business_name = df_business.business_name.values
N = len(u_h)
M = len(b_h)
for i in xrange(N):
    if numpy.count_nonzero(u_h[i, :]) == 0:
        continue
    user_id = uid[i]
    user_business_cosine = []
    categories1, categories2, categories3 = categories[u_h[i, :].argsort()[:3][::-1]]
    for j in numpy.random.randint(0, M, size=100):
        if numpy.count_nonzero(b_h[j, :]) == 0:
            user_business_cosine.append(1)
            continue
        user_business_cosine.append(cosine(u_h[i,:], b_h[j, :]))
    business_number = numpy.array(user_business_cosine).argsort()[:3][::-1]
    business1, business2, business3 = business_name[business_number]
    cursor.execute("""INSERT INTO business_recommend3 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",
                   (user_id,
                    i, 
                    business1,
                    business2,
                    business3,
                    categories1,
                    categories2,
                    categories3))
    if i % 10 ==0:
        print i
db.commit()

NameError: name 'df_user' is not defined